# Visualize Binary File as Image

In [1]:
import sys
import os
import hashlib
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
from cryptography.fernet import Fernet

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
# see https://gist.github.com/cbwar/d2dfbc19b140bd599daccbe0fe925597
def sizeof_fmt(num, suffix='b'):
    """Readable file size
    :param num: Bytes value
    :type num: int
    :param suffix: Unit suffix (optionnal) default = o
    :type suffix: str
    :rtype: str
    """
    for unit in ['', 'k', 'M', 'G', 'T', 'P', 'E', 'Z']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

BUF_SIZE = 65536  # lets read data in 64kb chunks!

def show_input_file(input_file):
    md5 = hashlib.md5()
    sha1 = hashlib.sha1()

    with open(input_file, 'rb') as f:
        while True:
            data = f.read(BUF_SIZE)
            if not data:
                break
            md5.update(data)
            sha1.update(data)

    md5 = md5.hexdigest()
    sha1 = sha1.hexdigest()

    print("file", input_file)
    print("MD5: {0}".format(md5))
    print("SHA1: {0}".format(sha1))

    arr = np.fromfile(input_file, dtype=np.ubyte)

    filesize = len(arr)
    filesize_text = sizeof_fmt(filesize)
    print("size:", filesize_text)

    linelength = math.ceil(math.sqrt(len(arr)))
    len_missing = (linelength**2 - len(arr))

    # add zero padding so we have proper square
    arr_padded = np.pad(arr, (0, len_missing), mode="constant", constant_values=0)
    del arr

    # reshape into quadratic matrix
    matrix = arr_padded.reshape(linelength, linelength)
    del arr_padded

    input_filename = os.path.basename(input_file)
    output_filename = input_filename + ".png"

    # see https://www.geeksforgeeks.org/matplotlib-pyplot-pcolormesh-in-python/
    fig, ax = plt.subplots()
    ax.matshow(matrix)
    del matrix

    ax.set_xlabel("SHA1: " + sha1 + "\nMD5: " + md5)
    #plt.ylim(0, 1000)  # Set the y-axis limits
    #plt.xlim(0, 1000)  # Set the x-axis limits
    plt.title(input_filename + " (" + sizeof_fmt(filesize) + ")", loc="left", fontweight="bold")
    plt.show()

In [ ]:
input_jpg = "C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_images\ost_campus_buchs.jpg"
show_input_file(input_jpg)

In [ ]:
input_png = "C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_images\ost_campus_buchs.png"
show_input_file(input_png)

In [ ]:
input_gif = "C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_images\ost_campus_buchs.gif"
show_input_file(input_gif)

In [ ]:
input_bmp = "C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_images\ost_campus_buchs.bmp"
show_input_file(input_bmp)

In [ ]:
unprotected_excel = r'C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_files\unprotected_excel.xlsx'
show_input_file(unprotected_excel)

In [ ]:
protected_excel = r"C:\Git\SecurityDataVisualization\Visualize Binary File as Image\example_files\protected_excel.xlsx"
show_input_file(protected_excel)

In [ ]:
# Generate a random encryption key
encryption_key = Fernet.generate_key()

# Create a Fernet cipher object with the encryption key
cipher = Fernet(encryption_key)

# Read the document content
with open(protected_excel, 'rb') as file:
    document_content = file.read()

# Encrypt the document content
encrypted_content = cipher.encrypt(document_content)

file_location = os.path.dirname(protected_excel)

# Write the encrypted content to a new file
with open(str(file_location + '\encrypted.xlsx'), 'wb') as file:
    file.write(encrypted_content)

In [ ]:
encrypted_file = str(file_location + '\encrypted.xlsx')
show_input_file(encrypted_file)